In [1]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

In [2]:
import html5lib
print(html5lib.__version__)

1.1


In [11]:
standings_url = "https://fbref.com/en/comps/676/European-Championship-Stats#all_Group%20stage"
data= requests.get(standings_url)
data

<Response [429]>

In [4]:
from io import StringIO

In [12]:
years = list(range(2024,2019, -4))
years


[2024, 2020]

In [6]:
all_matches = []

In [7]:

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text, 'html.parser')
    standings_table = soup.select('div.section_content', id='div_Group Stage')[0] 
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        try:
        # Use StringIO to wrap the HTML content
            matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]
            if not matches.empty:
                soup = BeautifulSoup(data.text, 'html.parser')
                links = [l.get("href") for l in soup.find_all('a')]
                links = [l for l in links if l and 'all_comps/shooting/' in l]
        
                if links:
                    data = requests.get(f"https://fbref.com{links[0]}")
                    try:
                        # Use StringIO to wrap the HTML content
                        shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]
                        if not shooting.empty:
                            shooting.columns = shooting.columns.droplevel()
                            try:
                                team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "PK", "PKatt"]], on="Date")
                                team_data = team_data[team_data["Comp"].isin(["UEFA Euro Qualifying", "UEFA Euro"])]
                                team_data["Season"] = year
                                team_data["team"] = team_name
                                all_matches.append(team_data)
                            except ValueError as e:
                                print(f"Error merging data for {team_name}: {e}")
                                continue
                        else:
                            print(f"No shooting data found for {team_name}")
                    except ValueError as e:
                        print(f"Error reading shooting data for {team_name}: {e}")
                else:
                    print(f"No shooting data link found for {team_name}")
            else:
                 print(f"No Scores n fixtures data found for {team_name}")

        except ValueError as e:
                                print(f"Error merging data for {team_name}: {e}")
                                continue
        
        time.sleep(1.5)

Error reading shooting data for Hungary Men: No tables found
Error merging data for Scotland Men: No tables found
Error merging data for Spain Men: No tables found
Error merging data for Italy Men: No tables found
Error merging data for Croatia Men: No tables found
Error merging data for Albania Men: No tables found
Error merging data for England Men: No tables found
Error merging data for Denmark Men: No tables found
Error merging data for Slovenia Men: No tables found
Error merging data for Serbia Men: No tables found
Error merging data for Austria Men: No tables found
Error merging data for France Men: No tables found
Error merging data for Netherlands Men: No tables found
Error merging data for Poland Men: No tables found
Error merging data for Romania Men: No tables found
Error merging data for Belgium Men: No tables found
Error merging data for Slovakia Men: No tables found
Error merging data for Ukraine Men: No tables found
Error merging data for Portugal Men: No tables found
Er

IndexError: list index out of range

In [13]:
years = list(range(2019,2015, -4))
years
standings_url = "https://fbref.com/en/comps/676/2021/2021-European-Championship-Stats"
data= requests.get(standings_url)
data

<Response [429]>

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text, 'html.parser')
    standings_table = soup.select('div.section_content', id='div_Group Stage')[0] 
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        try:
        # Use StringIO to wrap the HTML content
            matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]
            if not matches.empty:
                soup = BeautifulSoup(data.text, 'html.parser')
                links = [l.get("href") for l in soup.find_all('a')]
                links = [l for l in links if l and 'all_comps/shooting/' in l]
        
                if links:
                    data = requests.get(f"https://fbref.com{links[0]}")
                    try:
                        # Use StringIO to wrap the HTML content
                        shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]
                        if not shooting.empty:
                            shooting.columns = shooting.columns.droplevel()
                            try:
                                team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "PK", "PKatt"]], on="Date")
                                team_data = team_data[team_data["Comp"].isin(["UEFA Euro Qualifying", "UEFA Euro"])]
                                team_data["Season"] = year
                                team_data["team"] = team_name
                                all_matches.append(team_data)
                            except ValueError as e:
                                print(f"Error merging data for {team_name}: {e}")
                                continue
                        else:
                            print(f"No shooting data found for {team_name}")
                    except ValueError as e:
                        print(f"Error reading shooting data for {team_name}: {e}")
                else:
                    print(f"No shooting data link found for {team_name}")
            else:
                 print(f"No Scores n fixtures data found for {team_name}")

        except ValueError as e:
                                print(f"Error merging data for {team_name}: {e}")
                                continue
        
        time.sleep(1.5)

In [8]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df.to_csv("matches.csv")                     

In [9]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,referee,match report,notes,sh,sot,dist,pk,pkatt,season,team
4,2024-06-14,21:00,UEFA Euro,Group stage,Fri,Home,W,5.0,1.0,sct Scotland,...,Clément Turpin,Match Report,NaN,19.0,9.0,20.7,1,1,2024,Germany Men
5,2024-06-19,18:00,UEFA Euro,Group stage,Wed,Home,W,2.0,0.0,hu Hungary,...,Danny Makkelie,Match Report,NaN,19.0,7.0,17.8,0,0,2024,Germany Men
6,2024-06-23,21:00,UEFA Euro,Group stage,Sun,Away,D,1.0,1.0,ch Switzerland,...,Daniele Orsato,Match Report,NaN,18.0,3.0,15.9,0,0,2024,Germany Men
0,2023-03-25,18:00,UEFA Euro Qualifying,Group stage,Sat,Away,W,5.0,0.0,by Belarus,...,Alejandro Hernández,Match Report,NaN,14.0,6.0,NaN,0,0,2024,Switzerland Men
1,2023-03-28,20:45,UEFA Euro Qualifying,Group stage,Tue,Home,W,3.0,0.0,il Israel,...,Nikola Dabanović,Match Report,NaN,16.0,9.0,NaN,0,0,2024,Switzerland Men
2,2023-06-16,20:45,UEFA Euro Qualifying,Group stage,Fri,Away,W,2.0,1.0,ad Andorra,...,Balázs Berke,Match Report,NaN,13.0,8.0,NaN,0,0,2024,Switzerland Men
3,2023-06-19,20:45,UEFA Euro Qualifying,Group stage,Mon,Home,D,2.0,2.0,ro Romania,...,Daniele Orsato,Match Report,NaN,17.0,6.0,NaN,0,0,2024,Switzerland Men
4,2023-09-09,20:45,UEFA Euro Qualifying,Group stage,Sat,Away,D,2.0,2.0,xk Kosovo,...,Jakob Kehlet,Match Report,NaN,13.0,4.0,NaN,0,0,2024,Switzerland Men
5,2023-09-12,20:45,UEFA Euro Qualifying,Group stage,Tue,Home,W,3.0,0.0,ad Andorra,...,Elçin Məsiyev,Match Report,NaN,22.0,3.0,NaN,1,1,2024,Switzerland Men
6,2023-10-15,18:00,UEFA Euro Qualifying,Group stage,Sun,Home,D,3.0,3.0,by Belarus,...,João Pinheiro,Match Report,NaN,24.0,7.0,NaN,0,0,2024,Switzerland Men
